In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from SyntheticDataModule import *
from estimators import *
from utils import *

In [ ]:
with open('exp_configs/sce2.json', 'r') as file:
    try:
        jD = json.load(file)
    except json.JSONDecodeError:
        print("Invalid JSON format in the input file.")

RCTData = SyntheticDataModule(jD['save_df'], jD['cov_dim'], jD['rct_size'], 0, jD['RCT']['px_dist'], jD['RCT']['px_args'], jD['RCT']['prop_fn'], jD['RCT']['prop_args'], jD['RCT']['tte_params'])
OSData = SyntheticDataModule(jD['save_df'], jD['cov_dim'], jD['rct_size'] * jD['maxm'], 1, jD['OS']['px_dist'], jD['OS']['px_args'], jD['OS']['prop_fn'], jD['OS']['prop_args'], jD['OS']['tte_params'])

df_rct_oracle, df_rct = RCTData.get_df()
df_os_oracle, df_os = OSData.get_df()
df_combined = pd.concat([df_rct, df_os], axis=0, ignore_index=True)  # merge the dataframes into one

summary_df = pd.concat([RCTData.summary(plot=True), OSData.summary(plot=True)], axis=0, ignore_index=True)
summary_df

,S,d,n,True mean Y0,Impute mean Y0,Drop mean Y0,Oracle-IPCW mean Y0,True mean Y1,Impute mean Y1,Drop mean Y1,Oracle-IPCW mean Y1,True ATE,Impute ATE,Drop ATE,Oracle-IPCW ATE
0,0,2,1000,1.710546,1.688682,1.695300,1.724458,5.714593,5.477020,5.504795,5.302640,4.004047,3.788338,3.809495,3.578183
1,1,2,2000,1.641082,1.644962,1.638831,1.700727,5.382113,3.663191,4.956591,5.804245,3.741031,2.018229,3.317760,4.103519


In [ ]:
# x_space = np.linspace(-10,10,401)
# cov_name = 'X1'
# os_oracle_prop = OSData.calc_oracle_prop(x_space, cov_name)
# plt.figure()
# plt.plot(x_space, os_oracle_prop)
# plt.xlabel(cov_name)
# plt.ylabel(f'P(A=1|{cov_name},S=1)')
# plt.title(f'Oracle propensity score in study S=1 wrt covariate {cov_name}')
# plt.show()

# t = np.linspace(0,10,101)
# cov_vals = [0, 0]
# tbs_Y0 = RCTData.get_oracle_surv_curve(t, cov_vals, 'Y0')
# tbs_Y1 = RCTData.get_oracle_surv_curve(t, cov_vals, 'Y1')
# tbs_C0 = RCTData.get_oracle_surv_curve(t, cov_vals, 'C0')
# tbs_C1 = RCTData.get_oracle_surv_curve(t, cov_vals, 'C1')
# plt.figure()
# plt.plot(t, tbs_Y0, label='Y0')
# plt.plot(t, tbs_Y1, label='Y1')
# plt.plot(t, tbs_C0, label='C0')
# plt.plot(t, tbs_C1, label='C1')
# plt.xlabel('t')
# plt.ylabel(r'$S(t)$')
# plt.title(f'True survival curves in study S=0 with X={cov_vals}')
# plt.legend()
# plt.show()

In [ ]:
a = np.load('results/sce1/pvals.npy')